In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow import keras

In [26]:
# Load datasets
combats = pd.read_csv("datasets/combats.csv")
pokemon = pd.read_csv("datasets/pokemon.csv")
test_data = pd.read_csv("datasets/tests.csv")

In [27]:
# Create a dictionary mapping Pokémon ID to their stats and types
pokemon_dict = pokemon.set_index("#").to_dict(orient="index")

In [28]:
# Pokémon type effectiveness matrix
type_chart = {
    "Normal": {"Rock": 0.5, "Ghost": 0, "Steel": 0.5},
    "Fire": {"Fire": 0.5, "Water": 0.5, "Grass": 2, "Ice": 2, "Bug": 2, "Rock": 0.5, "Dragon": 0.5, "Steel": 2},
    "Water": {"Fire": 2, "Water": 0.5, "Grass": 0.5, "Ground": 2, "Rock": 2, "Dragon": 0.5},
    "Electric": {"Water": 2, "Electric": 0.5, "Grass": 0.5, "Ground": 0, "Flying": 2, "Dragon": 0.5},
    "Grass": {"Fire": 0.5, "Water": 2, "Grass": 0.5, "Poison": 0.5, "Ground": 2, "Flying": 0.5, "Bug": 0.5, "Rock": 2, "Dragon": 0.5, "Steel": 0.5},
    "Ice": {"Fire": 0.5, "Water": 0.5, "Ice": 0.5, "Grass": 2, "Ground": 2, "Flying": 2, "Dragon": 2, "Steel": 0.5},
    "Fighting": {"Normal": 2, "Ice": 2, "Rock": 2, "Dark": 2, "Steel": 2, "Poison": 0.5, "Flying": 0.5, "Psychic": 0.5, "Bug": 0.5, "Fairy": 0.5, "Ghost": 0},
    "Poison": {"Grass": 2, "Poison": 0.5, "Ground": 0.5, "Rock": 0.5, "Ghost": 0.5, "Steel": 0},
    "Ground": {"Fire": 2, "Electric": 2, "Grass": 0.5, "Poison": 2, "Flying": 0, "Bug": 0.5, "Rock": 2, "Steel": 2},
    "Flying": {"Electric": 0.5, "Grass": 2, "Fighting": 2, "Bug": 2, "Rock": 0.5, "Steel": 0.5},
    "Psychic": {"Fighting": 2, "Poison": 2, "Psychic": 0.5, "Dark": 0, "Steel": 0.5},
    "Bug": {"Fire": 0.5, "Grass": 2, "Fighting": 0.5, "Poison": 0.5, "Flying": 0.5, "Psychic": 2, "Ghost": 0.5, "Dark": 2, "Steel": 0.5, "Fairy": 0.5},
    "Rock": {"Fire": 2, "Ice": 2, "Fighting": 0.5, "Ground": 0.5, "Flying": 2, "Bug": 2, "Steel": 0.5},
    "Ghost": {"Normal": 0, "Psychic": 2, "Dark": 0.5, "Ghost": 2},
    "Dragon": {"Dragon": 2, "Steel": 0.5, "Fairy": 0},
    "Dark": {"Fighting": 0.5, "Psychic": 2, "Dark": 0.5, "Fairy": 0.5},
    "Steel": {"Fire": 0.5, "Water": 0.5, "Electric": 0.5, "Ice": 2, "Rock": 2, "Steel": 0.5, "Fairy": 2},
    "Fairy": {"Fighting": 2, "Poison": 0.5, "Steel": 0.5, "Dark": 2, "Dragon": 2}
}

In [29]:
# Function to calculate type effectiveness
def get_type_effectiveness(attacker_type1, attacker_type2, defender_type1, defender_type2):
    effectiveness = 1.0
    for atk_type in [attacker_type1, attacker_type2]:
        if pd.isna(atk_type):
            continue
        for def_type in [defender_type1, defender_type2]:
            if pd.isna(def_type):
                continue
            effectiveness *= type_chart.get(atk_type, {}).get(def_type, 1.0)
    return effectiveness

In [30]:
# Feature engineering
def get_features(row):
    p1 = pokemon_dict[row["First_pokemon"]]
    p2 = pokemon_dict[row["Second_pokemon"]]

    type_effectiveness = get_type_effectiveness(p1["Type 1"], p1["Type 2"], p2["Type 1"], p2["Type 2"])

    return [
        p1["HP"], p1["Attack"], p1["Defense"], p1["Sp. Atk"], p1["Sp. Def"], p1["Speed"], int(p1["Legendary"]),
        p2["HP"], p2["Attack"], p2["Defense"], p2["Sp. Atk"], p2["Sp. Def"], p2["Speed"], int(p2["Legendary"]),
        type_effectiveness
    ]

In [31]:
# Prepare training data
X = combats.apply(get_features, axis=1).tolist()
y = (combats["Winner"] == combats["First_pokemon"]).astype(int)  # 1 if First_pokemon wins, else 0


In [32]:
# Convert to NumPy arrays
X = np.array(X)
y = np.array(y)

In [33]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
# Train the Decision Tree model
model = DecisionTreeClassifier(max_depth=10, random_state=42)
model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=10, random_state=42)

In [35]:
# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")

Model Accuracy: 0.9435


In [36]:
# Prepare test predictions
test_features = np.array(test_data.apply(get_features, axis=1).tolist())
test_predictions = model.predict(test_features)

In [37]:
# Save results
test_data["Winner"] = test_data["First_pokemon"] * test_predictions + test_data["Second_pokemon"] * (1 - test_predictions)
test_data.to_csv("test_predictions.csv", index=False)
print("Predictions saved to test_predictions.csv")

Predictions saved to test_predictions.csv


In [38]:
# Save the model as an H5 file
keras_model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

keras_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
keras_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
# Save model architecture to JSON
model_json = keras_model.to_json()
with open("pokemon_battle_model.json", "w") as json_file:
    json_file.write(model_json)


print("Model architecture saved as pokemon_battle_model.json")

keras_model.save("pokemon_battle_model.h5")
print("H5 model saved as pokemon_battle_model.h5")

Epoch 1/10


C:\Users\Jan\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 2s 752us/step - accuracy: 0.7795 - loss: 0.9329 - val_accuracy: 0.8574 - val_loss: 0.3785
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 657us/step - accuracy: 0.8804 - loss: 0.3445 - val_accuracy: 0.9158 - val_loss: 0.2755
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 597us/step - accuracy: 0.9112 - loss: 0.2659 - val_accuracy: 0.9351 - val_loss: 0.2261
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 738us/step - accuracy: 0.9228 - loss: 0.2345 - val_accuracy: 0.9194 - val_loss: 0.2266
Epoch 5/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 623us/step - accuracy: 0.9270 - loss: 0.2248 - val_accuracy: 0.9181 - val_loss: 0.2304
Epoch 6/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 632us/step - accuracy: 0.9285 - loss: 0.2175 - val_accuracy: 0.9080 - val_loss: 0.2367
Epoch 7/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 620us/step - accuracy: 0.9262 - loss: 0.2167 - val_accuracy: 0.9361 - val_loss: 0.2200
Epoch 8/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 1s 627us/step - accuracy: 0.9327 - loss: 0.20

Model architecture saved as pokemon_battle_model.json
H5 model saved as pokemon_battle_model.h5


In [39]:
# Predict on test data using H5 model
loaded_model = keras.models.load_model("pokemon_battle_model.h5")
test_features = np.array(test_data.apply(get_features, axis=1).tolist())
test_predictions = (loaded_model.predict(test_features) > 0.5).astype(int)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step


In [40]:
test_data["Winner"] = test_data["First_pokemon"] * test_predictions.flatten() + test_data["Second_pokemon"] * (1 - test_predictions.flatten())
test_data.to_csv("test_predictions.csv", index=False)
print("Predictions saved to test_predictions.csv")

Predictions saved to test_predictions.csv
